In [1]:
# Generic
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings, gc
warnings.filterwarnings("ignore")

# Tensorflow
import tensorflow as tf

# ktrain
import ktrain
from ktrain import text

# sklearn
from sklearn.model_selection import train_test_split

In [3]:
from tensorflow import keras
tf.__version__

'2.10.1'

In [4]:
df = pd.read_csv("Training Data For the Topic- Environment .csv", header='infer')
# Dropping Null Values
df.dropna(inplace=True)

# Total Records
print("Total Records: ", df.shape[0])

# Inspect
df.head()


Total Records:  10441


,Topic,Sentence
0,Environment,collaborating with a range of collaborating wi...
1,No,"their elders , past and present , but perhaps..."
2,Environment,factor in our wellbeing is nature or the natu...
3,Environment,percent of the world 's disease is attributab...
4,Environment,million deaths per year globally and 1.1 bill...


In [8]:
# Data Split
target = ['Topic']
data = ['Sentence']

X = df[data]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(9396, 1) (1045, 1) (9396, 1) (1045, 1)


In [9]:
# Common Parameters
max_len = 500
batch_size = 16
learning_rate = 5e-5
epochs = 5

In [10]:
#########
## With Transformer = bert-base-uncased
######
# Transformer Model
model_ = 'bert-base-uncased'
t_mod = text.Transformer(model_, maxlen=500, classes = ['No','Environment'])


'''Converting split data to list [so it can processed]'''
#train
X_tr = X_train['Sentence'].tolist()
y_tr = y_train['Topic'].tolist()

#test
X_ts = X_test['Sentence'].tolist()
y_ts = y_test['Topic'].tolist()


# Pre-processing training & test data
train = t_mod.preprocess_train(X_tr,y_tr)
test = t_mod.preprocess_train(X_ts,y_ts)

# Model Classifier
model = t_mod.get_classifier()

learner = ktrain.get_learner(model, train_data=train, val_data=test, batch_size=batch_size)

preprocessing train...
language: en
train sequence lengths:
	mean : 51
	95percentile : 67
	99percentile : 118


Is Multi-Label? False
preprocessing train...
language: en
train sequence lengths:
	mean : 51
	95percentile : 67
	99percentile : 115


Is Multi-Label? False


In [11]:
# Train Model
learner.fit_onecycle(learning_rate, epochs)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/5
588/588 [==============================] - 47611s 81s/step - loss: 0.2260 - accuracy: 0.9126 - val_loss: 0.0254 - val_accuracy: 0.9933
Epoch 2/5
588/588 [==============================] - 50153s 85s/step - loss: 0.0212 - accuracy: 0.9950 - val_loss: 0.0225 - val_accuracy: 0.9962
Epoch 3/5
588/588 [==============================] - 47171s 80s/step - loss: 0.0172 - accuracy: 0.9964 - val_loss: 0.0281 - val_accuracy: 0.9962
Epoch 4/5
588/588 [==============================] - 47214s 80s/step - loss: 0.0084 - accuracy: 0.9984 - val_loss: 0.0194 - val_accuracy: 0.9962
Epoch 5/5
588/588 [==============================] - 46475s 79s/step - loss: 0.0019 - accuracy: 0.9994 - val_loss: 0.0222 - val_accuracy: 0.9962


In [13]:
# Evaluate
x = learner.validate(class_names=t_mod.get_classes())

33/33 [==============================] - 777s 24s/step
              precision    recall  f1-score   support

 Environment       0.99      0.99      0.99       195
          No       1.00      1.00      1.00       850

    accuracy                           1.00      1045
   macro avg       0.99      0.99      0.99      1045
weighted avg       1.00      1.00      1.00      1045



In [14]:
# to save the fine tuned model
ktrain.get_predictor(learner.model,preproc=t_mod).save('EnvironmentBinaryClassifier2')
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)
predictor.save("./ktrain/EnvironmentClassifier2")

In [15]:
# add the name of the patient stories
df1=pd.read_excel("Patient Segments as one data entry for Bert Characterisation .xlsx")

classes = ['Environment','']
predictor = ktrain.get_predictor(learner.model, preproc=t_mod)

segmentId=df1['SegmentId']
segList=list(segmentId)

startTime=df1['StartTime']
strList=list(startTime)

endTime=df1['EndTime']
endList=list(endTime)

transcript=df1['Transcript']
transList=list(transcript)
#transList1=[]
#for i in transList:
#    if i != 'nan':
#        transList1.append(i)

In [14]:
# I do not need this part as the patient segments' transcript is one line cell
i=0
segTranscript=[]
allSegmentsTrans=[]
startime=strList[0]
endTime=endList[0]
segTrigger=i
while i < len(segList)-2:
    #print(segList[i])
    #print()
    if segList[i]==segList[i+1]:
        print(segList[i],'  ==  ',segList[i+1])
        print()
        segTranscript.append(transList[i])
        #segTranscript.append(transList1[i+1])
        print( transList[i])
        #print( transList1[i+1])
        print()
        i+=1
    else:
        print(segList[i],'  !=  ',segList[i+1])
        #print()
        #print('start time= ',strList[i+1],'    ','end time= ',endList[i+1])

        segTranscript.append(transList[i])
        listToString=' '.join([str(item) for item in segTranscript ])
        allSegmentsTrans.append([segList[i],startime,endTime,segTranscript,listToString])
        print(listToString)
        print()
        segTranscript=[]
        
        startime=strList[i+1]
        endTime=endList[i+1]
        
        i+=1

MxmrjLQvg7E1   !=   MxmrjLQvg7E2
cancer cancer and there's there's it's spread everywhere and there's nothing we can do. I remember I was holding on to my mum screaming, don't let me die and she mum was just going I won't you're, not die and i was going. just don't let me

MxmrjLQvg7E2   !=   MxmrjLQvg7E3
 that I completely lost my vision. So everything just went black and i suffered with horrific migraines. So i got kind of i didn't got rushed into hospital, but i went for an eye test first and at my test at boots. That'S where

MxmrjLQvg7E3   !=   MxmrjLQvg7E4
 there's nothing we can do. it's spread all over. The body and she needs and my mom said staying in This hospital, to for her days. We want to take her home so a family, they brought me home and i had 24-hour care. I had

MxmrjLQvg7E4   !=   MxmrjLQvg7E5
 and now, i'm on in a it's like. I tell myself that it's like. it's i'm like. it's just it's like. chronic illness. There'S people with ms there's people that there's people th

HCgXqdpQGhc23   !=   HCgXqdpQGhc24
 You know you do n't feel any of that other than the stick of the iv , but you know it 's again kind of stressful because you 're sitting there all by yourself waiting for your scan um scan itself . Is you know relatively

HCgXqdpQGhc24   !=   HCgXqdpQGhc25
 people : um ivy that wa a lot for for you to swallow . Of course , at that point hearing stage four lung cancer - um wa , your husband there with you . Do you remember how you were able to

HCgXqdpQGhc25   !=   HCgXqdpQGhc26
 and you know it wa a school day and then my kid came home from school . So we did n't want them to know what wa going on yet because again i 've just been diagnosed . I do n't know what my treatment is . I do n't know what 's

HCgXqdpQGhc26   !=   HCgXqdpQGhc27
 going to happen so we kind of put on you know like a fake smile on our face and went around on with you know a regular day , um it wa almost surreal . I mean i i have to say i mean i literally do n't remember a lot ab

In [24]:
#### 
# Write the annotation result to Excel File- old videos

import xlrd
import xlsxwriter
workbook = xlsxwriter.Workbook('Health Domain-PatientStories-BinaryClassifier Bert-Environment2.xlsx')
worksheet = workbook.add_worksheet('Sheet1')
worksheet.write('A1', 'SegmentId')
worksheet.write('B1', 'StartTime')
worksheet.write('C1', 'EndTime')
worksheet.write('D1', 'Transcript')
worksheet.write('E1', 'Topic')
row=1
col=0
transRow=0

for i in range(0,len(segList)):
    
    print(segList[i])
    
    worksheet.write(row,col,str(segList[i]))
    worksheet.write(row,col+1,str(strList[i]))
    worksheet.write(row,col+2,str(endList[i]))
    worksheet.write(row,col+3,str(transList[i]))
    
    print('transc ',transList[i],' type ',type(transList[i]))
    print()
    if str(transList[i])=='nan':
        transList[i]='no'
        worksheet.write(row,col+4,predictor.predict(transList[i]))
    else:
        worksheet.write(row,col+4,predictor.predict(transList[i]))
        
    row+=1
    col=0
        
        

    
workbook.close()   


MxmrjLQvg7E1
transc  Two doctors came up to me with their Two doctors came up to me with their their laptop in front and they just closed the curtain, and i just remember thinking that time when they closed the curtain, This is this, isn't going to be good and he just looked at me and said: it's  type  <class 'str'>

1/1 [==============================] - 0s 183ms/step
MxmrjLQvg7E2
transc   cancer and there's there's it's spread everywhere and there's nothing we can do. I remember I was holding on to my mum screaming, don't let me die and she mum was just going I won't you're, not die and i was going. just don't let me  type  <class 'str'>

1/1 [==============================] - 0s 181ms/step
MxmrjLQvg7E3
transc   die mum probably the reason why i'm here today is down to the biopsy that I had of my back, which, which, is where they found the alk mutation and the treatment that was available to me and without it i probably wouldn't be here today  type  <class 'str'>

1/1 [==============

1/1 [==============================] - 0s 193ms/step
rQutyErqBG012
transc   that 's not true , because you have options in your treatments . Dr.Xu ! Listen ! You can live longer , we have things . I always thought that anybody when chemo throws up all the time - and that was very scary . I have a fairly high pain threshold . I have a very low noise  type  <class 'str'>

1/1 [==============================] - 0s 179ms/step
rQutyErqBG013
transc   special and I thought I 'm never gon na do chemo , they 're , never gon na . Do that to me , and I was n't going to do it and then I thought I 'm kind of role model I might to my children that I wo n't even try and once you know you could always stop it . You should at least give it a try and  type  <class 'str'>

1/1 [==============================] - 0s 208ms/step
rQutyErqBG014
transc   see if it 's that terrible and it was n't , they give you anti-nausea medicine , they give you steroids , I do n't feel like eating , but I do n't feel like th

1/1 [==============================] - 0s 161ms/step
hQQ5xQGZxdA9
transc   i'm here. for It um. dave thanks, for sharing that about you um. would love to maybe just jump right in then, if you're ready, um, you know, if you could talk about just what were the first symptoms or feelings that made you  type  <class 'str'>

1/1 [==============================] - 0s 184ms/step
hQQ5xQGZxdA10
transc   realize? Hey something's, not right here? Oh, it really started um. i was a young guy. I was a non-smoker and i was up in maine with a with a friend of mine fishing. In may was freezing cold. I remember this, but but  type  <class 'str'>

1/1 [==============================] - 0s 175ms/step
hQQ5xQGZxdA11
transc   i got super sick. While i was up there, i was the chills and i came back and it and it turns out that i that i had pneumonia um i went so i went in and you know, got saw my doctor and and got the chest x-ray got. I got the antibiotics  type  <class 'str'>

1/1 [=========================

1/1 [==============================] - 0s 165ms/step
hQQ5xQGZxdA37
transc   just in terms of what to and we'll i think that i mean what the way i understood at the time. You know that this, that The The chest x-ray couldn't couldn't define what was so The The ct scan. was a logical step  type  <class 'str'>

1/1 [==============================] - 0s 174ms/step
hQQ5xQGZxdA38
transc   don't think that i'd ever had one that so um. i i asked my You know I had a meeting with my doctor to you know, to talk about The steps so he was kind of like my quarterback,  type  <class 'str'>

1/1 [==============================] - 0s 160ms/step
hQQ5xQGZxdA39
transc   think is also when people are diagnosed. trying to find That That That thinking about your care right it's like for me it was going to be at general hospital because that's my  type  <class 'str'>

1/1 [==============================] - 0s 163ms/step
hQQ5xQGZxdA40
transc   primary care was but You know When my mother experience and she was

1/1 [==============================] - 0s 164ms/step
hQQ5xQGZxdA66
transc   thing you want to say, about this, part of your we'll what I would say and We probably talk about this, but i you know, there are ways to connect. with other patients now i think and that's what I would  type  <class 'str'>

1/1 [==============================] - 0s 166ms/step
hQQ5xQGZxdA67
transc   encourage people do if you get that initial. I'Ve been diagnosed. Oh my gosh, what do I do and and and that process. of setting. up a care team and those is it community setting. is it  type  <class 'str'>

1/1 [==============================] - 0s 166ms/step
hQQ5xQGZxdA68
transc   academic setting and um. who can Who who's been through the same thing that i'm that i'm anticipating who can help, okay, great so with that said uh. were over to The next  type  <class 'str'>

1/1 [==============================] - 0s 168ms/step
hQQ5xQGZxdA69
transc   section. The next where dave. will talk about surgery and The recovery

...

1/1 [==============================] - 0s 163ms/step
O5DOre3MFlw8
transc   to the hospital where they pronounced me dead . That happened quite a few time over the rest of my lifetime I thought I wa in hell . Because it wa horrible I mean It really wa .  type  <class 'str'>

1/1 [==============================] - 0s 161ms/step
O5DOre3MFlw9
transc   It wa horrible and it wa scary and I thought I 'd died and gone to hell . What wa horrible about it ? Well I could n't breathe . I could n't talk . I could n't move . If a fly crawled up my leg I could n't stop it . I could n't scratch it .  type  <class 'str'>

1/1 [==============================] - 0s 197ms/step
O5DOre3MFlw10
transc   I could n't kill it . I could n't I could n't tell 'them I needed to go to the bathroom . So I lived in it . Interviewer When you were in the hospital for two year were you around other kid who had polio ? I wa put I wa put in a big  type  <class 'str'>

1/1 [==============================] - 0s 192ms/step
O5D

1/1 [==============================] - 0s 196ms/step
O5DOre3MFlw35
transc   So I kept living and It changed all the time . But it wa fun . I enjoyed it . I enjoyed the struggle . I enjoyed the challenge . I enjoyed the people .  type  <class 'str'>

1/1 [==============================] - 0s 160ms/step
O5DOre3MFlw36
transc   Was there a point where you could no longer breathe outside of the iron lung ? Well it 's a conscious thing . I 'm having to think about breathing okay . I had to think about Paul take another breath . Okay gulping noise Like that . okay . And uh  type  <class 'str'>

1/1 [==============================] - 0s 178ms/step
O5DOre3MFlw37
transc   it take a lot of energy . I always determined to go on to college and when I got through the application stage they laughed at me and said What are you talking about ? Well I showed them what I wa talking about . It took me 2 year to convince  type  <class 'str'>

1/1 [==============================] - 0s 185ms/step
O5DOre3MFlw

1/1 [==============================] - 0s 172ms/step
O5DOre3MFlw61
transc   and want to operate in a day to day environment . Just like you do . Daniel This is the worm gear . And what it doe is is it transfer the energy from the motor to a circular motion that then connects to  type  <class 'str'>

1/1 [==============================] - 0s 183ms/step
O5DOre3MFlw62
transc   this bar and that actual connection back there is a is a shifter of a Model T Ford Model T that wa that the shifter for it . And that 's what they used to to actually make the connection there . Cause that wa a spare part that they could get ahold of cause they were in a desperate need to get part fast .  type  <class 'str'>

1/1 [==============================] - 0s 189ms/step
O5DOre3MFlw63
transc   Interviewer Do you have any movement of your body ? No I have no movement . I ca n't do anything . So I need complete care . This is my baby . That 's what I call her . I call her my baby because she took care of me for

1/1 [==============================] - 0s 179ms/step
bqkquNb5Z8A15
transc   that I have a new found appreciation for . soft piano music Being able just to take a walk around my block . Just being able to drive up to my sister 's house if I wan na see the kid . I feel like I 'm a brand new me . And thanks to Penn Medicine  type  <class 'str'>

1/1 [==============================] - 0s 183ms/step
bqkquNb5Z8A16
transc   I can live my life again .  type  <class 'str'>

1/1 [==============================] - 0s 104ms/step
qsAnlV8DUXo1
transc   My name is lorenzo . I live in oak cliff . This is my home hi , i 'm reid . I'Ve been with lorenzo for 23 year , i 'm an artist to a lot of different kind of medium artwork from glass fusing to  type  <class 'str'>

1/1 [==============================] - 0s 178ms/step
qsAnlV8DUXo2
transc   painting and ceramic . I remember when all of this started kind of happening , i wa playing volleyball and got to a point where i wa playing that i could n't catch 

1/1 [==============================] - 0s 199ms/step
_4ehrdRPEqE2
transc   research nurse and I work in breast oncology so , prior to being a lung cancer survivor you 'd order test for patient and you did it because that 's standard of care , that 's driven per protocol . That 'S just what we do with your disease .  type  <class 'str'>

1/1 [==============================] - 0s 184ms/step
_4ehrdRPEqE3
transc   However . Now I actually understand what people are going through when they 're getting those test , so you can completely relate to what they are going through . If there 's a silver lining to this story , I had tested positive for a mutation of  type  <class 'str'>

1/1 [==============================] - 0s 182ms/step
_4ehrdRPEqE4
transc   which there wa targeted therapy medicine that are already out there and it wa an egfr mutation that I tested positive for so park because you can park and then you just like Rest , okay , now put it into reverse  type  <class 'str'>

1/1 [===